### Word Group In Youtube

In [206]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [207]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [208]:
# language pair
lang_folder = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # native word end index 28,200,1000,5000,10000, 20000, 40000

# word ratio
if word_end == 28 or word_end == 200:
    unique_word_ratio = 70
    include_wordgroup_ratio = 70
elif word_end == 1000 or word_end == 5000:
    unique_word_ratio = 80
    include_wordgroup_ratio = 80
elif word_end == 10000:
    unique_word_ratio = 85
    include_wordgroup_ratio = 85
else:
    unique_word_ratio = 90
    include_wordgroup_ratio = 90

# word usage
if word_end == 28 or word_end == 200 or word_end == 1000 or word_end == 5000 or word_end == 10000:
    word_usage_min = 10
    word_usage_max = 100
elif word_end == 20000:
    word_usage_min = 7
    word_usage_max = 100
else:
    word_usage_min = 5
    word_usage_max = 100

# use for manuel set
#unique_word_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 ; 10000 = 85 ; 20000,40000 = 90
#include_wordgroup_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 : 10000 = 85 ; 20000,40000 = 90
#word_usage_min = 10  # 28,200,1000,5000,10000 = 10 ; 20000 = 7 ; 40000 = 5
#word_usage_max = 100


# sentence ratio and time shift
adjust_sent_word_ratio = 15  #28,200 = 15 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
#adjust_text_word_ratio = 5  #28,200 = 5 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
shift = 0.3  # sentence time shift

# func select
if word_end == 28:
    sentence_word_count_number = 2
else:
    sentence_word_count_number = 5   
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
                                  # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"word_end:{word_end}\nfile_ext:{file_ext}\nunique_word_ratio:{unique_word_ratio}\n\
include_wordgroup_ratio:{include_wordgroup_ratio}\nword_usage_min:{word_usage_min}\nword_usage_max:{word_usage_max}")

word_end:28
file_ext:6
unique_word_ratio:70
include_wordgroup_ratio:70
word_usage_min:10
word_usage_max:100


In [209]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Data/1-Word Group In Youtube Sentence"

Path(path).mkdir(parents=True, exist_ok=True)

In [210]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by=f"{target_opt_column}", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [211]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [212]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [213]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [214]:
def repetation_ratio(text):
    '''
    repetation_ratio(text) 
    text is a like as sentence and def compute repetation word ratio\n
    repetation_ratio("yok bir şey yok bir şey yok bir şey") 
    '''
    word_list = word_tokenize(text)
    word_num = len(word_list)
    word_unique_num = len(set(word_list))
    ratio = int((word_unique_num/word_num)*100)
    
    return ratio

In [215]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,de,16588988
1,je,16386475
2,pas,11547876
3,le,10592792
4,la,9939090
...,...,...
317922,puisetil,5
317923,shouzheng,5
317924,steinam,5
317925,cochezle,5


In [216]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [217]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,de,16588988
1,je,16386475
2,pas,11547876
3,le,10592792
4,la,9939090
5,que,9858145
6,vous,8980559
7,tu,8776452
8,et,8039570
9,un,7421102


In [218]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [219]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [220]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

28

#### Youtube Sentence Analysis

In [221]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.000,00:00:00.200,perrosguirec bretagne,BU7DYGMSUA4
1,00:00:30.972,00:00:33.609,located in the regional natural park of armorica,4nmWfnmVgo4
2,00:00:34.263,00:00:36.866,the huelgoat forest,4nmWfnmVgo4
3,00:00:37.315,00:00:39.315,has always inspired popular stories,4nmWfnmVgo4
4,00:00:40.132,00:00:43.640,places with mysterious names sometimes refer t...,4nmWfnmVgo4
...,...,...,...,...
3010839,00:04:49.620,00:04:54.420,a noter quen cas de décès du titulaire du plan...,Wq8plZLZdzE
3010840,00:04:54.420,00:04:57.540,il ne fait donc pas lobjet dune transmission a...,Wq8plZLZdzE
3010841,00:04:57.540,00:05:01.800,vous souhaitez en savoir plus sur le traitemen...,Wq8plZLZdzE
3010842,00:05:01.800,00:05:05.160,dans ce cas nhésitez pas visiter notre site fi...,Wq8plZLZdzE


In [222]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [223]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,0.000,0.200,perrosguirec bretagne,BU7DYGMSUA4
1,1,30.972,33.609,located in the regional natural park of armorica,4nmWfnmVgo4
2,2,34.263,36.866,the huelgoat forest,4nmWfnmVgo4
3,3,37.315,39.315,has always inspired popular stories,4nmWfnmVgo4
4,4,40.132,43.640,places with mysterious names sometimes refer t...,4nmWfnmVgo4
...,...,...,...,...,...
3010839,3010839,289.620,294.420,a noter quen cas de décès du titulaire du plan...,Wq8plZLZdzE
3010840,3010840,294.420,297.540,il ne fait donc pas lobjet dune transmission a...,Wq8plZLZdzE
3010841,3010841,297.540,301.800,vous souhaitez en savoir plus sur le traitemen...,Wq8plZLZdzE
3010842,3010842,301.800,305.160,dans ce cas nhésitez pas visiter notre site fi...,Wq8plZLZdzE


In [224]:
#df_youtube_sentence[df_youtube_sentence["index"] == 1287503]

In [225]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/(len(sent_word)+0.001))*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [226]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.000,0.200,perrosguirec bretagne,BU7DYGMSUA4,0.0,"perrosguirec, bretagne",
1,1,30.972,33.609,located in the regional natural park of armorica,4nmWfnmVgo4,0.0,"park, the, in, of, regional, located, armorica...",
2,2,34.263,36.866,the huelgoat forest,4nmWfnmVgo4,0.0,"forest, the, huelgoat",
3,3,37.315,39.315,has always inspired popular stories,4nmWfnmVgo4,0.0,"always, inspired, stories, popular, has",
4,4,40.132,43.640,places with mysterious names sometimes refer t...,4nmWfnmVgo4,0.0,"arthur, sometimes, refer, to, with, king, plac...",
...,...,...,...,...,...,...,...,...
3010839,3010839,289.620,294.420,a noter quen cas de décès du titulaire du plan...,Wq8plZLZdzE,20.0,"titulaire, pea, a, noter, clôturé, plan, décès...","est, de, le"
3010840,3010840,294.420,297.540,il ne fait donc pas lobjet dune transmission a...,Wq8plZLZdzE,30.0,"donc, dune, fait, transmission, aux, lobjet, h...","ne, pas, il"
3010841,3010841,297.540,301.800,vous souhaitez en savoir plus sur le traitemen...,Wq8plZLZdzE,23.1,"souhaitez, traitement, pea, france, sur, savoi...","en, vous, le"
3010842,3010842,301.800,305.160,dans ce cas nhésitez pas visiter notre site fi...,Wq8plZLZdzE,33.3,"site, visiter, nhésitez, fiscaloofr, notre, cas","ce, pas, dans"


In [227]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.000,0.200,perrosguirec bretagne,BU7DYGMSUA4,0.0,"perrosguirec, bretagne",NaN
1,1,30.972,33.609,located in the regional natural park of armorica,4nmWfnmVgo4,0.0,"park, the, in, of, regional, located, armorica...",NaN
2,2,34.263,36.866,the huelgoat forest,4nmWfnmVgo4,0.0,"forest, the, huelgoat",NaN
3,3,37.315,39.315,has always inspired popular stories,4nmWfnmVgo4,0.0,"always, inspired, stories, popular, has",NaN
4,4,40.132,43.640,places with mysterious names sometimes refer t...,4nmWfnmVgo4,0.0,"arthur, sometimes, refer, to, with, king, plac...",NaN
...,...,...,...,...,...,...,...,...
3010839,3010839,289.620,294.420,a noter quen cas de décès du titulaire du plan...,Wq8plZLZdzE,20.0,"titulaire, pea, a, noter, clôturé, plan, décès...","est, de, le"
3010840,3010840,294.420,297.540,il ne fait donc pas lobjet dune transmission a...,Wq8plZLZdzE,30.0,"donc, dune, fait, transmission, aux, lobjet, h...","ne, pas, il"
3010841,3010841,297.540,301.800,vous souhaitez en savoir plus sur le traitemen...,Wq8plZLZdzE,23.1,"souhaitez, traitement, pea, france, sur, savoi...","en, vous, le"
3010842,3010842,301.800,305.160,dans ce cas nhésitez pas visiter notre site fi...,Wq8plZLZdzE,33.3,"site, visiter, nhésitez, fiscaloofr, notre, cas","ce, pas, dans"


In [228]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,18,15.09,20.61,est un éloge des eaux dormantes une halte migr...,BYd2A4JSdOo,35.7,"eaux, halte, oiseaux, dormantes, éloge, migrat...","des, et, est, un, une"
1,19,20.61,26.55,deau herbiers et roselières sont le refuge des...,BYd2A4JSdOo,23.1,"herbiers, sont, ou, despèces, menacées, roseli...","et, en, le"
2,20,26.55,31.02,de rivière brochet ou butor étoilé au fil de l...,BYd2A4JSdOo,20.0,"étoilé, au, ou, brochet, surgit, butor, rivièr...","la, de, un"
3,21,31.02,36.24,chargé dhistoire rappelant par sa toponymie le...,BYd2A4JSdOo,16.7,"toponymie, rappelant, viromandui, gaulois, cha...","des, le"
4,22,36.24,42.75,mentionné par césar dans la guerre des gaules ...,BYd2A4JSdOo,25.0,"mentionné, passé, antique, césar, gaules, guer...","la, ce, des, dans"
...,...,...,...,...,...,...,...,...
2561173,3010839,289.62,294.42,a noter quen cas de décès du titulaire du plan...,Wq8plZLZdzE,20.0,"titulaire, pea, a, noter, clôturé, plan, décès...","est, de, le"
2561174,3010840,294.42,297.54,il ne fait donc pas lobjet dune transmission a...,Wq8plZLZdzE,30.0,"donc, dune, fait, transmission, aux, lobjet, h...","ne, pas, il"
2561175,3010841,297.54,301.80,vous souhaitez en savoir plus sur le traitemen...,Wq8plZLZdzE,23.1,"souhaitez, traitement, pea, france, sur, savoi...","en, vous, le"
2561176,3010842,301.80,305.16,dans ce cas nhésitez pas visiter notre site fi...,Wq8plZLZdzE,33.3,"site, visiter, nhésitez, fiscaloofr, notre, cas","ce, pas, dans"


In [229]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

<ipython-input-229-ca15520b3270>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
0,18,15.09,20.61,est un éloge des eaux dormantes une halte migr...,BYd2A4JSdOo,35.7,"eaux, halte, oiseaux, dormantes, éloge, migrat...","des, et, est, un, une",est un
2,20,26.55,31.02,de rivière brochet ou butor étoilé au fil de l...,BYd2A4JSdOo,20.0,"étoilé, au, ou, brochet, surgit, butor, rivièr...","la, de, un",de la
4,22,36.24,42.75,mentionné par césar dans la guerre des gaules ...,BYd2A4JSdOo,25.0,"mentionné, passé, antique, césar, gaules, guer...","la, ce, des, dans",dans la
6,24,48.81,53.94,probablement remanié lors de la conquête romai...,BYd2A4JSdOo,23.1,"position, probablement, lors, conquête, romain...","la, de, il",de la
7,26,58.35,64.17,et les fossés sont par ailleurs bien visibles ...,BYd2A4JSdOo,16.7,"visibles, sont, ailleurs, incursions, barbares...","et, les",et les
...,...,...,...,...,...,...,...,...,...
2561168,3010833,269.40,272.70,dans ce cas les gains réalisés sur le pea depuis,Wq8plZLZdzE,40.0,"depuis, pea, gains, sur, réalisés, cas","ce, le, les, dans",dans ce
2561169,3010835,275.76,278.40,il importe peu à cet égard que le décès soit i...,Wq8plZLZdzE,27.3,"soit, peu, cet, à, décès, importe, égard, inte...","que, il, le",que le
2561170,3010836,278.40,282.60,dans les cinq ans de louverture du plan ou apr...,Wq8plZLZdzE,26.7,"délai, louverture, cinq, après, ans, ou, plan, du","ce, de, les, dans",dans les
2561174,3010840,294.42,297.54,il ne fait donc pas lobjet dune transmission a...,Wq8plZLZdzE,30.0,"donc, dune, fait, transmission, aux, lobjet, h...","ne, pas, il",il ne


In [230]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,est un,15.090000,15.544588,est un éloge des eaux dormantes une halte migr...,BYd2A4JSdOo
1,de la,28.961447,29.373158,de rivière brochet ou butor étoilé au fil de l...,BYd2A4JSdOo
2,dans la,37.528438,38.138750,mentionné par césar dans la guerre des gaules ...,BYd2A4JSdOo
3,de la,50.336786,50.764286,probablement remanié lors de la conquête romai...,BYd2A4JSdOo
4,et les,58.350000,58.915833,et les fossés sont par ailleurs bien visibles ...,BYd2A4JSdOo
...,...,...,...,...,...
1306456,dans ce,269.400000,269.950000,dans ce cas les gains réalisés sur le pea depuis,Wq8plZLZdzE
1306457,que le,277.031111,277.422222,il importe peu à cet égard que le décès soit i...,Wq8plZLZdzE
1306458,dans les,278.400000,278.947826,dans les cinq ans de louverture du plan ou apr...,Wq8plZLZdzE
1306459,il ne,294.420000,294.742759,il ne fait donc pas lobjet dune transmission a...,Wq8plZLZdzE


In [231]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,est un,14.790000,15.844588,est un éloge des eaux dormantes une halte migr...,BYd2A4JSdOo
1,de la,28.661447,29.673158,de rivière brochet ou butor étoilé au fil de l...,BYd2A4JSdOo
2,dans la,37.228438,38.438750,mentionné par césar dans la guerre des gaules ...,BYd2A4JSdOo
3,de la,50.036786,51.064286,probablement remanié lors de la conquête romai...,BYd2A4JSdOo
4,et les,58.050000,59.215833,et les fossés sont par ailleurs bien visibles ...,BYd2A4JSdOo
...,...,...,...,...,...
1306456,dans ce,269.100000,270.250000,dans ce cas les gains réalisés sur le pea depuis,Wq8plZLZdzE
1306457,que le,276.731111,277.722222,il importe peu à cet égard que le décès soit i...,Wq8plZLZdzE
1306458,dans les,278.100000,279.247826,dans les cinq ans de louverture du plan ou apr...,Wq8plZLZdzE
1306459,il ne,294.120000,295.042759,il ne fait donc pas lobjet dune transmission a...,Wq8plZLZdzE


In [232]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,est un,15,16,est un éloge des eaux dormantes une halte migr...,BYd2A4JSdOo
1,de la,29,30,de rivière brochet ou butor étoilé au fil de l...,BYd2A4JSdOo
2,dans la,37,38,mentionné par césar dans la guerre des gaules ...,BYd2A4JSdOo
3,de la,50,51,probablement remanié lors de la conquête romai...,BYd2A4JSdOo
4,et les,58,59,et les fossés sont par ailleurs bien visibles ...,BYd2A4JSdOo
...,...,...,...,...,...
1306456,dans ce,269,270,dans ce cas les gains réalisés sur le pea depuis,Wq8plZLZdzE
1306457,que le,277,278,il importe peu à cet égard que le décès soit i...,Wq8plZLZdzE
1306458,dans les,278,279,dans les cinq ans de louverture du plan ou apr...,Wq8plZLZdzE
1306459,il ne,294,295,il ne fait donc pas lobjet dune transmission a...,Wq8plZLZdzE


In [233]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,pour que vous mais pas mais pas pour que ça,3150,3154,que vous réfléchissiez pour que vous mais pas ...,9dieGtqlzXc
1,et cest pour ça que cest pour ça que le de,323,325,était à la solde de kgb oui jignorais really e...,w0Q3Yr9RzAE
2,cest pour ça que est ce que cest pour ça,2705,2708,cest pour ça que est ce que cest pour ça,dmjms0MkbnE
3,cest pour ça que cest pour ça que je me,1274,1278,donc cest pour ça que cest pour ça que je me s...,_c13bmndeAM
4,cest que ça cest pas cest pas ça qui,2903,2905,si cest que ça cest pas cest pas ça qui va te ...,PX4cGSnHBUs
...,...,...,...,...,...
1306456,de la,472,472,acquérir cest davoir de la largeur parce que c...,oACvYI5h0Q8
1306457,de la,205,206,dor sur laquelle on posait douze pains sacrés ...,98cIyH1cyyk
1306458,de la,241,242,table dor portant les pains de la présence et ...,98cIyH1cyyk
1306459,de la,271,272,dont le sang sauvait de la mort et dont la cha...,98cIyH1cyyk


In [234]:
df_word_group_time_loc_in_sent_result["unique_word_ratio"] = df_word_group_time_loc_in_sent_result["search_string"].apply(lambda text: repetation_ratio(text))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,pour que vous mais pas mais pas pour que ça,3150,3154,que vous réfléchissiez pour que vous mais pas ...,9dieGtqlzXc,60
1,et cest pour ça que cest pour ça que le de,323,325,était à la solde de kgb oui jignorais really e...,w0Q3Yr9RzAE,63
2,cest pour ça que est ce que cest pour ça,2705,2708,cest pour ça que est ce que cest pour ça,dmjms0MkbnE,60
3,cest pour ça que cest pour ça que je me,1274,1278,donc cest pour ça que cest pour ça que je me s...,_c13bmndeAM,60
4,cest que ça cest pas cest pas ça qui,2903,2905,si cest que ça cest pas cest pas ça qui va te ...,PX4cGSnHBUs,55
...,...,...,...,...,...,...
1306456,de la,472,472,acquérir cest davoir de la largeur parce que c...,oACvYI5h0Q8,100
1306457,de la,205,206,dor sur laquelle on posait douze pains sacrés ...,98cIyH1cyyk,100
1306458,de la,241,242,table dor portant les pains de la présence et ...,98cIyH1cyyk,100
1306459,de la,271,272,dont le sang sauvait de la mort et dont la cha...,98cIyH1cyyk,100


In [235]:
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result[df_word_group_time_loc_in_sent_result["unique_word_ratio"] > unique_word_ratio]
df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-235-d6186b820154>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
<ipython-input-235-d6186b820154>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,pas que nous est ce que cest que la,1956,1958,responsable de ça bien sûr mais il ny a pas qu...,W6PsDuxLVqQ
1,mais pas que et cest ce que je vous,17,19,ou comme codeuses pour la plupart mais pas que...,vKAZypYoHtk
2,et cest pour ça que cest ce que je,1207,1210,soi sous toutes ses formes cest juste crucial ...,Iw_c7byC3q8
3,pas pour vous mais pour nous cest,10,12,et je sais pas pour vous mais pour nous cest e...,H5SVeWnZo_A
4,cest vous et cest pour ça que je,51,53,cest vous et cest pour ça que je voulais être là,HRSfDp0j0GU
...,...,...,...,...,...
628658,je me,459,461,effectivement jai fait mes enquêtes et puis je...,JX9At3lttGQ
628659,de je,759,760,en plein milieu de je trouve ça formidable vou...,JX9At3lttGQ
628660,et en,229,229,cest quen agissant de cette manière en faisant...,nZ_zav40Rvo
628661,et ne,183,183,il appelle son agent ils découvrent que disney...,nZ_zav40Rvo


In [236]:
df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-236-e137816795b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)


,search_string,start_time,end_time,sentence,video_id,include_string_ratio
0,pas que nous est ce que cest que la,1956,1958,responsable de ça bien sûr mais il ny a pas qu...,W6PsDuxLVqQ,36
1,mais pas que et cest ce que je vous,17,19,ou comme codeuses pour la plupart mais pas que...,vKAZypYoHtk,34
2,et cest pour ça que cest ce que je,1207,1210,soi sous toutes ses formes cest juste crucial ...,Iw_c7byC3q8,37
3,pas pour vous mais pour nous cest,10,12,et je sais pas pour vous mais pour nous cest e...,H5SVeWnZo_A,47
4,cest vous et cest pour ça que je,51,53,cest vous et cest pour ça que je voulais être là,HRSfDp0j0GU,66
...,...,...,...,...,...,...
628658,je me,459,461,effectivement jai fait mes enquêtes et puis je...,JX9At3lttGQ,8
628659,de je,759,760,en plein milieu de je trouve ça formidable vou...,JX9At3lttGQ,7
628660,et en,229,229,cest quen agissant de cette manière en faisant...,nZ_zav40Rvo,5
628661,et ne,183,183,il appelle son agent ils découvrent que disney...,nZ_zav40Rvo,6


In [237]:
df_word_group_time_loc_include_string_ratio = df_word_group_time_loc_in_sent_result_repeat_ratio[df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] > include_wordgroup_ratio]
df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)
df_word_group_time_loc_include_string_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_include_string_ratio

<ipython-input-237-ea709f90da2b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,mais cest pas pour ça que ça,1180,1182,mais cest pas pour ça que ça va arriver,YuxMdNbmxzM
1,cest pour ça que je vous le,672,675,cest pour ça que je vous le montre,aDBLqp0bIPw
2,cest pour ça que je vous le,261,262,cest pour ça que je vous le dis,NuBCoiQLhHY
3,et cest pour ça que dans ce,3402,3405,et cest pour ça que dans ce forum,GWSsZ6KCuJI
4,cest pour ça que jai pas de,926,929,cest pour ça que jai pas de mec,KAgyTxbTNxw
...,...,...,...,...,...
3262,et en,52,55,et en,xFSnkgyicVY
3263,et en,83,86,et en,gwUSvIV0CoA
3264,et la,295,299,et la,d3VH6R81KDI
3265,et en,377,379,et en,d3VH6R81KDI


In [238]:
df_youtube_word_usage = word_usage_result(word_list, df_word_group_time_loc_include_string_ratio, "search_string", "video_id", word_usage_min, word_usage_max)
df_youtube_word_usage.drop_duplicates(inplace=True)
df_youtube_word_usage

,search_string,video_id
0,mais dans,zwMkA4dt8ig
1,qui est pas,zwMkA4dt8ig
2,et de ce que jai,zLh5sr83kRk
3,et ce qui est en,zL6_Ewwy8dE
4,pour que ce qui,zF_9n4Fb1Xw
...,...,...
552,vous les,0MY1DIfT92Q
553,en de,0HBZCZEKLdQ
554,et le la,0CY8xdTakyw
555,jai pas,0A1sr_fKKhw


In [239]:
df_youtube_word_usage_merge = pd.merge(df_word_group_time_loc_include_string_ratio,df_youtube_word_usage,how="inner",on=["search_string","video_id"])
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","video_id"],inplace=True)
df_youtube_word_usage_merge.reset_index(drop=True, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,mais cest pas pour ça que ça,1180,1182,mais cest pas pour ça que ça va arriver,YuxMdNbmxzM
1,cest pour ça que je vous le,672,675,cest pour ça que je vous le montre,aDBLqp0bIPw
2,cest pour ça que je vous le,261,262,cest pour ça que je vous le dis,NuBCoiQLhHY
3,et cest pour ça que dans ce,3402,3405,et cest pour ça que dans ce forum,GWSsZ6KCuJI
4,cest pour ça que jai pas de,926,929,cest pour ça que jai pas de mec,KAgyTxbTNxw
...,...,...,...,...,...
542,on en,21,28,on en,MPh2CI4pHgI
543,je ne,3035,3037,je ne,0tkM8oJxoLY
544,il je,414,418,il je,pRJ31AuY1Bk
545,de la,698,706,de la,cOd9lQMiPkk


In [240]:
df_youtube_word_usage_merge["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id,video_url
0,mais cest pas pour ça que ça,1180,1182,mais cest pas pour ça que ça va arriver,YuxMdNbmxzM,https://www.youtube.com/watch?v=YuxMdNbmxzM&t=...
1,cest pour ça que je vous le,672,675,cest pour ça que je vous le montre,aDBLqp0bIPw,https://www.youtube.com/watch?v=aDBLqp0bIPw&t=...
2,cest pour ça que je vous le,261,262,cest pour ça que je vous le dis,NuBCoiQLhHY,https://www.youtube.com/watch?v=NuBCoiQLhHY&t=...
3,et cest pour ça que dans ce,3402,3405,et cest pour ça que dans ce forum,GWSsZ6KCuJI,https://www.youtube.com/watch?v=GWSsZ6KCuJI&t=...
4,cest pour ça que jai pas de,926,929,cest pour ça que jai pas de mec,KAgyTxbTNxw,https://www.youtube.com/watch?v=KAgyTxbTNxw&t=...
...,...,...,...,...,...,...
542,on en,21,28,on en,MPh2CI4pHgI,https://www.youtube.com/watch?v=MPh2CI4pHgI&t=21s
543,je ne,3035,3037,je ne,0tkM8oJxoLY,https://www.youtube.com/watch?v=0tkM8oJxoLY&t=...
544,il je,414,418,il je,pRJ31AuY1Bk,https://www.youtube.com/watch?v=pRJ31AuY1Bk&t=...
545,de la,698,706,de la,cOd9lQMiPkk,https://www.youtube.com/watch?v=cOd9lQMiPkk&t=...


In [241]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(2000)
df_youtube_word_usage_merge.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.csv", index=False)

In [242]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(700)
df_youtube_word_usage_merge.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

In [243]:
#df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)
#df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [244]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

['French_Word_Group_Max_In_Youtube_Sentence_28_Word6.csv',
 'French_Word_Group_Max_In_Youtube_Sentence_28_Word6.xlsx']

In [245]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [246]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass